<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/neary_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Faissパッケージのインストール
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00


In [2]:
# ベクトルDBを利用する
import faiss

In [3]:
# Embeddingファイルを読み込み Google Driveから
# Google Driveをマウントする
from google.colab import drive
drive.mount('/content/drive')

# ファイル
embedding_file =  '/content/drive/MyDrive/PL自動割り当てEmbeddingファイル/embeddings_all.npy'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np

# ファイルからEmbeddingを読み込む
target_embeds = np.load(embedding_file)


In [5]:
# インデックス数
d = target_embeds.shape[1]
print(d)

# Faissのインデックス生成
index = faiss.IndexFlatL2(d)

# 対象テキストをインデックスに追加
index.add(target_embeds)

1536


In [6]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 11.0 MB/s eta 0:00:00


In [7]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [8]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

In [17]:
# テキストを読み込む
txt_file_path = "./法律AI研究報告書.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

In [18]:
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# numpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [19]:
import csv
# 作業推薦者を出す
def recommend_workers(query_embedding, csv_file, index, k=10):
    # 近傍探索の実行
    D, I = index.search(query_embedding, k)
    print(D)
    print(I)

    # 類似ファイルのインデックス値を取得
    similar_indices = I[0].tolist()

    # CSVファイルから既存のファイルパスと担当者IDを読み込む
    file_paths = []
    worker_ids = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # ヘッダーをスキップ
        for row in reader:
            file_paths.append(row[1])
            worker_ids.append(int(row[2]))

    # 類似ファイルの担当者IDを取得
    similar_worker_ids = [worker_ids[i] for i in similar_indices]

    # 担当者IDの出現回数をカウント
    worker_id_counts = {}
    for worker_id in similar_worker_ids:
        worker_id_counts[worker_id] = worker_id_counts.get(worker_id, 0) + 1

    # 出現回数の多い順に担当者IDを取得
    sorted_worker_ids = sorted(worker_id_counts, key=worker_id_counts.get, reverse=True)

    # 推薦する作業担当者のIDを取得
    recommended_worker_ids = sorted_worker_ids[:3]  # 上位3人を推薦

    return recommended_worker_ids

In [20]:
# 確認
csv_file = "./worked_files.csv"

recommended_worker_ids = recommend_workers(in_embeds, csv_file, index)
print("Recommended worker IDs:", recommended_worker_ids)

[[0.03189817 0.04887741 0.08837742 0.11160477 0.13243756 0.1446165
  0.18959308 0.22103558 0.24178305 0.25575137]]
[[458 459 475 492 487 483 484 497 462 469]]
Recommended worker IDs: [5, 3, 4]
